In [1]:
from bs4 import BeautifulSoup
import urllib
import re 

In [137]:
def search(query):
    address = "http://www.bing.com/search?q=%s" % (urllib.parse.quote_plus(query))

    getRequest = urllib.request.Request(address, None, {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:54.0) Gecko/20100101 Firefox/54.0'})

    urlfile = urllib.request.urlopen(getRequest)
    htmlResult = urlfile.read(20000000)
    urlfile.close()

    soup = BeautifulSoup(htmlResult)

    [s.extract() for s in soup('span')]
    unwantedTags = ['strong', 'cite']
    
    for tag in unwantedTags:
        for match in soup.findAll(tag):
            match.replaceWithChildren()

    results = soup.findAll('li', { "class" : "b_algo" })

    return results

def get_item_info(item):
    links = search("green alternatives to "+item)
    titles = []
    urls = []
    for link in links :
        urls.append(str(link.find('h2').find('a')['href']).replace(" ", " "))
        titles.append(str(link.find('h2').text).replace(" ", " "))
    ts =[]
    us =[]
    i = 0
    for t in titles:
        if any(char.isdigit() for char in t):
            ts.append(t)
            us.append(urls[i])
        i=i+1
    return ts, us

def isrelevant(title, item):
    b = False
    greenwords = ['green', 'eco-friendly', 'sustainable']
    for g in greenwords:
        if g in str.lower(title):
            b = True
    i = ""
    '''
    if item[len(item)-1] =='s':
        i =item[:len(item)-1]
    if i in str.lower(title):
        if b:
            return True
    return False
    '''
    return b
  
def get_relevants(titles, urls, item):
    ind = 0
    us = []
    ts = []
    for t in titles:
        if isrelevant(t, item):
            us.append(urls[ind])
            ts.append(titles[ind])
        ind = ind + 1
    return ts, us

def get_items_from_urls(urls):
    items = []
    for u in urls:
        getRequest = urllib.request.Request(u, None, {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:54.0) Gecko/20100101 Firefox/54.0'})

        urlfile = urllib.request.urlopen(getRequest)
        htmlResult = urlfile.read(2000000)
        urlfile.close()

        soup = BeautifulSoup(htmlResult)
        for h in soup.findAll('h2'):
            b = True
            elem = h
            while(b):
                temp = elem.findChild()
                if(temp==None):
                    b= False
                else:
                    elem = temp
                    
            if len(elem.text)<50:
                items.append(elem.text)
    return items

def filter_items(items):
    ites = []
    b = True
    keywords = ['more', 'support', 'about']
    for i in items:
        if '.' in i:
            ""
        else:
            for k in keywords:
                if k in str.lower(i):
                    b = False
            if b:
                ites.append(i)
            
    return ites

def get_search_results(item, name):
    links = search("Buy eco-friendly "+item+" "+name)
    titles = []
    urls = []
    for link in links :
        urls.append(str(link.find('h2').find('a')['href']).replace(" ", " "))
        titles.append(str(link.find('h2').text).replace(" ", " "))
    
    return titles, urls

def getWebItems(filtered, item):
    webitems = []
    for f in filtered:
        obj = {}
        t, u = get_search_results(f, item)
        obj['name'] = f
        obj['urls'] = u[:3]
        obj['titles'] = t[:3]
        webitems.append(obj)
    return webitems
        

def getWebData(item):
    t, u = get_item_info(item)
    titles, urls = get_relevants(t, u, item)
    its = get_items_from_urls(urls)
    f = filter_items(its)
    items = getWebItems(f, item)
    return items

In [138]:
getWebData("shoes")

[{'name': 'Fish Leather',
  'urls': ['https://www.etsy.com/market/fish_leather_shoes',
   'https://www.etsy.com/market/eco_friendly_shoes',
   'https://www.greenandgrowing.org/top-eco-friendly-sneakers-shoes/'],
  'titles': ['Fish leather shoes | Etsy',
   'Eco friendly shoes | Etsy',
   'Top 6 Eco Friendly Sneakers and Sports Shoes You Can Buy ...']},
 {'name': 'Cork Leather / Cork Fabric',
  'urls': ['https://www.alibaba.com/product-detail/Eco-friendly-natural-Natural-PU-Cork_62413514689.html',
   'https://puretreecork.com/products/cork-fabric',
   'https://www.amazon.com/ZAIONE-Natural-Leather-Lightweight-Meterial/dp/B07KS6HTY7'],
  'titles': ['Eco-friendly Natural Natural Pu Cork Leather Fabric For ...',
   'Cork Fabric | Eco-Friendly Cork Fabric | Puretree Cork ...',
   'Amazon.com: ZAIONE Soft Natural Color Real Cork Leather ...']},
 {'name': 'Ocean Leather (Kelp)',
  'urls': ['https://greenerideal.com/guides/7042-natural-alternatives-to-leather-shoes/',
   'https://www.shoes.com

[{'name': 'Fish Leather',
  'urls': ['Fish leather shoes | Etsy',
   '7 Eco-friendly Sneakers You Can Actually Get Active In ...',
   'Earth Leather Sale Up to 40% Off - Free Shipping - Shoes.com'],
  'titles': ['https://www.etsy.com/market/fish_leather_shoes',
   'https://consciousfashion.co/guides/eco-friendly-sneakers-running-shoes',
   'https://www.shoes.com/earth-leather/category_230629']},
 {'name': 'Cork Leather / Cork Fabric',
  'urls': ['Amazon.com: cork fabric',
   'Eco-friendly Natural Natural Pu Cork Leather Fabric For ...',
   'Amazon.com: ZAIONE Soft Natural Color Real Cork Leather ...'],
  'titles': ['https://www.amazon.com/cork-fabric/s?k=cork+fabric',
   'https://www.alibaba.com/product-detail/Eco-friendly-natural-Natural-PU-Cork_62413514689.html',
   'https://www.amazon.com/ZAIONE-Natural-Leather-Lightweight-Meterial/dp/B07KS6HTY7']},
 {'name': 'Ocean Leather (Kelp)',
  'urls': ['3 Natural Alternatives to Leather Shoes | Greener Ideal',
   'No Animals Harmed: Sustaina

In [49]:
def shop(query):
    address = "http://www.bing.com/shop?q=%s" % (urllib.parse.quote_plus(query))

    getRequest = urllib.request.Request(address, None, {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:54.0) Gecko/20100101 Firefox/54.0'})

    urlfile = urllib.request.urlopen(getRequest)
    htmlResult = urlfile.read(20000000)
    urlfile.close()

    soup = BeautifulSoup(htmlResult)

    results = soup.findAll('div', class_='br-wholeCardClickable br-card br-small br-ocCiteHvrItm')

    return results

In [118]:
def get_shop_data(item):
    shopdata = shop("Buy eco-friendly "+item)
    shopdata = []
    for l in links:
        data = {}
        try:
            data['image']='https://www.bing.com/'+l.findAll('img')[0]['src']
            data['seller'] = l.findAll('span', class_='br-sellersCite')[0].text
            data['price'] = l.findAll('div', class_='pd-price br-standardPrice promoted')[0].text
            try:
                data['item'] = l.findAll('span')[-1]['title']
            except:
                data['item'] = "shoes"
            data['url'] = l.findAll('a', class_='br-adClickOut')[0]['href']

            shopdata.append(data)
        except:
            print('DIDNT WORK')
            break
    return shopdata
    

In [121]:
shopdata= get_shop_data("shoes")

DIDNT WORK


In [120]:
links = shop("Buy eco-friendly "+'shoes')

In [115]:
len(shopdata)

8

8